# VERDICT PREDICTION

In [1]:
%%capture
!pip install -U sentence-transformers
!pip install --quiet underthesea

In [2]:
import torch
from transformers import AutoModel, AutoTokenizer , TrainingArguments, Trainer , AutoModelForSequenceClassification , AdamW , get_scheduler ,DataCollatorWithPadding
from datasets import load_dataset
from accelerate import Accelerator
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import precision_score, recall_score, f1_score
from datasets import Dataset, DatasetDict
from gensim.utils import simple_preprocess
from tqdm.auto import tqdm
import torch.nn as nn
import sentence_transformers
from sentence_transformers import SentenceTransformer, util
from underthesea import word_tokenize
from accelerate import Accelerator
import re
SEED = 1337

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
def preprocess_text(text: str) -> str:
    text = re.sub(r"['\",\.\?:\-!]", "", text)
    text = text.strip()
    text = " ".join(text.split())
    text = text.lower()
    return text
verdict2label = {"SUPPORTED" : 2 , "REFUTED" : 0 , "NEI" : 1}
label2verdict = {v : k for k , v in verdict2label.items()}

### RANKING TEST DATA

In [5]:
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
test_data = pd.read_json('/kaggle/input/private-test-ranking-6/PRIVATE DATA RANKING 6' , lines = True)
test_ds = Dataset.from_pandas(test_data)
test_ds

Dataset({
    features: ['context', 'claim', '__index_level_0__', 'new_context'],
    num_rows: 5396
})

In [6]:
def reduce_context (sample ) :
    return {'new_context' : ". ".join ( sample["new_context"].split ('. ')[:-5] ) }
test_ds = test_ds.map ( reduce_context  )

  0%|          | 0/5396 [00:00<?, ?ex/s]

In [7]:
test_ds[0]

{'context': 'Sáng 7/6, bà Nguyễn Thị Lệ Thanh, Giám đốc Sở Du lịch Khánh Hòa cho biết, đã có báo cáo về hoạt động du lịch trong đợt Festival Biển Nha Trang – Khánh Hòa 2023, với chủ đề “Khánh Hòa – Khát vọng phát triển”.\n\nLễ hội lần thứ 10 diễn ra từ ngày 3 đến 6/6/2023, tại TP Nha Trang với hơn 70 hoạt động đa dạng về nội dung và hình thức, phản ánh được những giá trị văn hóa của con người, tinh thần lao động hăng say, tình yêu biển đảo quê hương.\n\nTrong đó, có nhiều chương trình nghệ thuật, biểu diễn thời trang, nhạc hội, hay diễu hành áo dài với hơn 6.000 người tham gia, Lễ hội ẩm thực với 42 món ăn là tinh hoa ẩm thực yến sào…, đã thu hút đông đảo người dân, du khách tham quan, theo dõi.\n\nSân khấu được thiết kế bên bờ biển với quy mô lớn, với nhiều thiết bị công nghệ hiện đại hàng đầu thế giới. Điểm nhấn là màn biểu diễn của 1.653 drone light lớn nhất Việt Nam từ trước tới nay. 1.653 drone thể hiện cho thời gian năm 1653, là khởi đầu cho sự hình thành và phát triển của tỉnh K

# SEGMENT DATA

In [8]:
checkpoint = "vinai/phobert-base-v2"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
def segment_text (samples) :
    return {"new_context" : [word_tokenize(sample, format="text") for sample in samples["new_context"] ],
            "claim_for_verdict" : [word_tokenize(sample, format="text") for sample in samples["claim"] ]
           }

In [10]:
test_ds_segment = test_ds.map (segment_text , batched = True)

  0%|          | 0/6 [00:00<?, ?ba/s]

In [11]:
test_ds_segment[0]

{'context': 'Sáng 7/6, bà Nguyễn Thị Lệ Thanh, Giám đốc Sở Du lịch Khánh Hòa cho biết, đã có báo cáo về hoạt động du lịch trong đợt Festival Biển Nha Trang – Khánh Hòa 2023, với chủ đề “Khánh Hòa – Khát vọng phát triển”.\n\nLễ hội lần thứ 10 diễn ra từ ngày 3 đến 6/6/2023, tại TP Nha Trang với hơn 70 hoạt động đa dạng về nội dung và hình thức, phản ánh được những giá trị văn hóa của con người, tinh thần lao động hăng say, tình yêu biển đảo quê hương.\n\nTrong đó, có nhiều chương trình nghệ thuật, biểu diễn thời trang, nhạc hội, hay diễu hành áo dài với hơn 6.000 người tham gia, Lễ hội ẩm thực với 42 món ăn là tinh hoa ẩm thực yến sào…, đã thu hút đông đảo người dân, du khách tham quan, theo dõi.\n\nSân khấu được thiết kế bên bờ biển với quy mô lớn, với nhiều thiết bị công nghệ hiện đại hàng đầu thế giới. Điểm nhấn là màn biểu diễn của 1.653 drone light lớn nhất Việt Nam từ trước tới nay. 1.653 drone thể hiện cho thời gian năm 1653, là khởi đầu cho sự hình thành và phát triển của tỉnh K

# ADD SPECIAL TOKEN

In [12]:
CLS = tokenizer.special_tokens_map["cls_token"]
SEP = tokenizer.special_tokens_map["sep_token"]
def add_special_token (samples) :
    return {"input" : [  CLS + " " + claim + " " + SEP + " " + context + " " + SEP for claim , context in zip ( samples["claim_for_verdict"] , samples["new_context"] )  ]}
test_ds_input = test_ds_segment.map (add_special_token , batched = True )

  0%|          | 0/6 [00:00<?, ?ba/s]

In [13]:
test_ds_input[0]

{'context': 'Sáng 7/6, bà Nguyễn Thị Lệ Thanh, Giám đốc Sở Du lịch Khánh Hòa cho biết, đã có báo cáo về hoạt động du lịch trong đợt Festival Biển Nha Trang – Khánh Hòa 2023, với chủ đề “Khánh Hòa – Khát vọng phát triển”.\n\nLễ hội lần thứ 10 diễn ra từ ngày 3 đến 6/6/2023, tại TP Nha Trang với hơn 70 hoạt động đa dạng về nội dung và hình thức, phản ánh được những giá trị văn hóa của con người, tinh thần lao động hăng say, tình yêu biển đảo quê hương.\n\nTrong đó, có nhiều chương trình nghệ thuật, biểu diễn thời trang, nhạc hội, hay diễu hành áo dài với hơn 6.000 người tham gia, Lễ hội ẩm thực với 42 món ăn là tinh hoa ẩm thực yến sào…, đã thu hút đông đảo người dân, du khách tham quan, theo dõi.\n\nSân khấu được thiết kế bên bờ biển với quy mô lớn, với nhiều thiết bị công nghệ hiện đại hàng đầu thế giới. Điểm nhấn là màn biểu diễn của 1.653 drone light lớn nhất Việt Nam từ trước tới nay. 1.653 drone thể hiện cho thời gian năm 1653, là khởi đầu cho sự hình thành và phát triển của tỉnh K

# TOKENIZE

In [14]:
def tokenize (sample) :
    return tokenizer(sample ["input"] , truncation= True , max_length = 256)
test_ds_tokenize = test_ds_input.map (tokenize , batched = True )
test_ds_tokenize

  0%|          | 0/6 [00:00<?, ?ba/s]

Dataset({
    features: ['context', 'claim', '__index_level_0__', 'new_context', 'claim_for_verdict', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5396
})

In [15]:
test_train = test_ds_tokenize.remove_columns (['context', 'claim', 'new_context', 'input' ])

In [16]:
test_train = test_train.remove_columns (['claim_for_verdict'])

In [17]:
test_train

Dataset({
    features: ['__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5396
})

# MODEL FOR VERDICT PREDICTION

In [18]:
from transformers import (
    AutoModel, AutoConfig,
    AutoTokenizer, logging
)
config = AutoConfig.from_pretrained(checkpoint)
config.update({'output_hidden_states':True})


In [19]:
class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * (num_hidden_layers+1 - layer_start), dtype=torch.float)
            )

    def forward(self, all_hidden_states):
        all_layer_embedding = all_hidden_states[self.layer_start:, :, :, :]
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()
        return weighted_average

In [20]:
class CustomPhoBert(nn.Module):
    def __init__(self):
        super(CustomPhoBert, self).__init__()
        self.model = AutoModel.from_pretrained(checkpoint , config = config)
        self.tokenizer = AutoTokenizer.from_pretrained(checkpoint)

        self.layer_start = 9
        self.pooler = pooler = WeightedLayerPooling(
            config.num_hidden_layers,
            layer_start=self.layer_start, layer_weights=None
        )
        # self.att = nn.Sequential(
        #                     nn.Linear(768 , 384),
        #                     nn.Tanh(),
        #                     nn.Linear ( 384 , 1),
        #                     nn.Softmax(dim=1)
        #                     )
        self.drop = nn.Dropout (0.1)
        self.fc = nn.Linear (config.hidden_size, 3 )
        #self._init_weights(self.fc)
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def forward(self, **batch):


        outputs = self.model (**batch )

        all_hidden_states = torch.stack(outputs[2])



        weighted_pooling_embeddings = self.pooler(all_hidden_states)
        weighted_pooling_embeddings = weighted_pooling_embeddings[:, 0]

        drop_feature = self.drop (weighted_pooling_embeddings)
        logits = self.fc(weighted_pooling_embeddings)

        return logits


In [21]:
phobert1 = CustomPhoBert ().to(device)
phobert1.load_state_dict(torch.load("/kaggle/input/phobert/Phobert_ranking_1_fold2_early_stopping"))
phobert1.eval()

phobert2 = CustomPhoBert ().to(device)
phobert2.load_state_dict(torch.load("/kaggle/input/phobert/Phobert_ranking_1_fold4_early_stopping"))
phobert2.eval()

phobert3 = CustomPhoBert ().to(device)
phobert3.load_state_dict(torch.load("/kaggle/input/phobert/Phobert_ranking_1_fold5_early_stopping"))
phobert3.eval()


phobert4 = CustomPhoBert ().to(device)
phobert4.load_state_dict(torch.load("/kaggle/input/phobert/Phobert_ranking_1_fold_3_early_stopping"))
phobert4.eval()

phobert5 = CustomPhoBert ().to(device)
phobert5.load_state_dict(torch.load("/kaggle/input/phobert/phobert_ranking_1_fold_1_early_stopping"))
phobert5.eval()

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'rober

CustomPhoBert(
  (model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNo

In [22]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
test_dataloader = DataLoader (test_train, batch_size= 256 ,shuffle  = False,collate_fn = data_collator )
accelerator = Accelerator ()
test_dataloader = accelerator.prepare (test_dataloader)

In [23]:
_id = []
preds =[]
for batch in tqdm ( test_dataloader) :
    with torch.no_grad () :
        _id.append ( batch.pop ('__index_level_0__').to('cpu').numpy () )
        
        logits1 = phobert1.forward (**batch)
        logits2 = phobert2.forward (**batch)
        logits3 = phobert3.forward (**batch)
        logits4 = phobert4.forward (**batch)
        logits5 = phobert5.forward (**batch)
        
        logits = (logits1 + logits2 + logits3 + logits4 + logits5) / 5
        

        predicts = torch.argmax (logits, dim = -1).to('cpu').numpy()
        verdicts = [label2verdict[predict] for predict in predicts]
        preds.append (verdicts)
_id = np.concatenate (_id)
preds = np.concatenate (preds)

  0%|          | 0/22 [00:00<?, ?it/s]

In [24]:
# _id = test_ds["__index_level_0__"]

In [25]:
(_id != test_ds["__index_level_0__"] ).sum ()

0

# EVIDENCE PREDICTION

In [26]:
# load evidence 
evidence_ds = test_ds.add_column("verdict", preds)
evidence_ds

Dataset({
    features: ['context', 'claim', '__index_level_0__', 'new_context', 'verdict'],
    num_rows: 5396
})

In [27]:
evidences = [] 
for sample in evidence_ds  :
    print (sample['__index_level_0__'])
    # NEU NHƯ NEI thì pred sẽ là "" luôn
    if (sample["verdict"] == "NEI") :
        pred = ""
    else :
        pred = sample['new_context'].split('. ')[0]
    
   
    evidences.append (pred)
    #KIỂM TRA ID, VỚI LẠI KIỂM TRA NEI

39773
37657
40265
47711
39930
37208
38194
42930
40100
44099
39986
37620
40772
47629
38199
43495
43746
47556
37625
38181
40287
44257
37406
45930
37411
45773
42756
47466
39806
42666
42912
37983
37463
43215
44003
39569
44061
43441
37475
39891
36900
43461
39840
40266
38555
38276
38232
43121
37375
43949
44203
37446
37342
39826
37871
39488
37349
42826
39506
42668
37923
37192
43370
40400
43120
39580
37062
44055
39988
42884
45901
37152
39676
40042
39442
43233
44048
40074
43941
39760
39709
37739
43584
37825
44183
37094
42690
43778
39445
38176
39911
44068
37592
39807
37353
38596
39551
38125
47729
38546
37976
38559
44245
43690
40817
43655
47595
40183
43596
38377
42990
43791
37485
37126
40883
44073
38278
38618
36877
44042
38554
47580
44159
40222
43333
38041
38129
43377
42771
44085
38542
43270
39498
43395
42886
40219
43592
37159
39639
43160
38296
40165
38269
43255
37219
37815
44207
42877
38247
45951
39458
43638
40057
37837
43367
42844
40010
38342
37048
37347
43915
37112
37732
43339
43065
39885
3861

In [28]:
# def compute_evidence (sample):
#     if 
#     return {"evidence" : sample['new_context'].split('. ')[0]}
# ds_evidence = test_ds.map (compute_evidence)
# evidences = ds_evidence["evidence"]

In [29]:
(evidence_ds["__index_level_0__"] != _id ).sum()

0

# SUBMISSION

In [30]:
import json
# Assuming you have the data dictionary as described
data = {
    "id": _id,
    "predicted_verdict": preds,
    "predicted_evidence": evidences,
}

In [31]:
data

{'id': array([39773, 37657, 40265, ..., 37834, 43741, 42819]),
 'predicted_verdict': array(['NEI', 'REFUTED', 'NEI', ..., 'REFUTED', 'SUPPORTED', 'SUPPORTED'],
       dtype='<U9'),
 'predicted_evidence': ['',
  'một số người khuyên em theo công nghệ thực phẩm thay vì công nghệ dệt may nhưng em ghét hóa',
  '',
  'nhãn hàng sẽ cùng đối tác đề ra những chiến lược phát triển kinh doanh đem lại sản phẩm chất lượng cho thị trường',
  'chúng tôi gọi đó là bụi thông minh và chỉ vài tế bào vi khuẩn cũng có thể tạo ra đủ năng lượng cho chúng',
  'người mắc bệnh phổi tắc nghẽn mạn tính hen suyễn bệnh màng phổi hội chứng giảm thông khí ngưng thở khi ngủ',
  'cdc khuyến nghị người trưởng thành người già và những người dễ mắc covid19 vẫn tiêm chủng đầy đủ theo lịch trình trước đó',
  'ngoài ra hiu cũng công bố chương trình học bổng doanh nghiệp đồng hành 2023',
  '',
  'cảng hoàn thành năm 2012 công suất thiết kế 280000 tấn hàng hóa và 440000 khách mỗi năm',
  'đa số cực dương làm từ than chì và đư

In [32]:
result = {}
for i in range ( len (data["id"]) ) :
    result [str (data["id"][i] ) ] = { "verdict": data ["predicted_verdict"][i], "evidence": data['predicted_evidence'][i]}

In [33]:
file_path = "/kaggle/working/public_result.json"
with open(file_path, 'w') as json_file:
    json.dump(result, json_file)

print("Data saved to", file_path)

Data saved to /kaggle/working/public_result.json


In [ ]:
### LUU Lai test_DS
test_ds.to_json ("test_data.json")